<a href="https://colab.research.google.com/github/JayThibs/transformers-from-scratch/blob/main/pytorch_lightning_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers with PyTorch Lightning

References:

* [Simple PyTorch Transformer Example with Greedy Decoding](https://colab.research.google.com/drive/1swXWW5sOLW8zSZBaQBYcGQkQ_Bje_bmI) by Sergey Karayev from Full Stack Deep Learning
* [The Annotated Transformer ++](https://github.com/gordicaleksa/pytorch-original-transformer/blob/main/The%20Annotated%20Transformer%20%2B%2B.ipynb) by gordicaleksa / The AI Epiphany
* [Transformers from Scratch](https://e2eml.school/transformers.html) by End-to-End ML School
* [Notes on GPT-2 and BERT models](https://www.kaggle.com/residentmario/notes-on-gpt-2-and-bert-models) by Aleksey Bilogur
* [GPT-3: Language Models are Few-Shot Learners (Paper Explained)](https://www.youtube.com/watch?v=SY5PvZrJhLE) by Yannic Kilcher
* [Various Annotated Transformer PyTorch Papers](https://nn.labml.ai/transformers/index.html) by labml.ai

This notebook provides a simple, self-contained example of Transformer models:

* using both the encoder and decoder parts
* greedy decoding at inference time

For the first part of the notebook, we'll train on a simple synthetic example, and use PyTorch Lightning since it will greatly simplify the training loop.

When the first transformer paper came out (Attention Is All You Need), the authors used the transformer architecture for machine translation. This means that they needed both the encoder and decoder parts of the architecture to first encode the text, and then decode (generate) the translation.

After that paper, researchers realized that they could use the encoder and decoder separately in order to create models for approaching different tasks. This led to the emergence of BERT-like models (encoder / non-autoregressive) and GPT-like models (decoder / autoregressive).

However, we'll be going over each part of the entire transformer.

Note: Autoregressive means that model only takes into account the text or context that came before our current prediction. Each new prediction is taken into account in the next prediction. Non-autoregressive models take the entire surrounding context! So, a model like BERT uses bi-directionality (that's what the B stands for) and takes in the entire surrounding context for word prediction when trying to predict a masked word. This makes it so that GPT is great at generating text, while BERT is great at taking in an entire piece of text and classifying it.

# Installations

In [2]:
!pip install pytorch_lightning spacy --quiet

# Imports

In [6]:
# Python native libs
import math
import copy
import os
import time
import enum
import argparse

# Visualization imports
import matplotlib.pyplot as plt
import seaborn


# Deep learning imports
import pytorch_lightning as pl
import torch
import torch.nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
from torch.hub import download_url_to_file

# Data manipulation
import numpy as np
# from torchtext.data import Dataset, BucketIterator, Field, Example
from torchtext.data.utils import interleave_keys
from torchtext import datasets
# from torchtext.data import Example
import spacy

# BLEU
from nltk.translate.bleu_score import corpus_bleu